_API Reference: https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#steps_

In [1]:
!pip install sagemaker transformers==4.44.2 --quiet

In [2]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep, ProcessingStep, CreateModelStep, CacheConfig
from sagemaker.workflow.parameters import ParameterString
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.model_step import ModelStep
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sess = sagemaker.Session()

sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## 取得 Training Datasets

In [3]:
import json
import boto3

# 初始化 S3 客戶端，來源區域是 ap-northeast-1，目標區域是 us-west-2
s3_source = boto3.client('s3', region_name="us-west-2")
s3_target = boto3.client('s3', region_name="us-west-2")

def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key

def copy_s3_object(source_uri, target_bucket):
    source_bucket, source_key = parse_s3_uri(source_uri)
    try:
        # 從來源 bucket 下載檔案
        response = s3_source.get_object(Bucket=source_bucket, Key=source_key)
        file_content = response['Body'].read()

        # 將檔案上傳到目標 bucket
        s3_target.put_object(Bucket=target_bucket, Key=source_key, Body=file_content)
        print(f"Copied {source_key} to {target_bucket}")
    except Exception as e:
        print(f"Error copying {source_key}: {str(e)}")

# s3 URI
base_uri = 's3://aws-educate-09-28-sagemaker-workshop-oregon/datasets/phi-3/'
train_uri = base_uri + 'train_dataset.json'
test_uri = base_uri + 'test_dataset.json'

# 你的目標 S3 bucket
target_bucket = sess.default_bucket()


# 複製 train 和 test 資料到新的 S3 bucket
copy_s3_object(train_uri, target_bucket)
copy_s3_object(test_uri, target_bucket)

datasets_s3_uri = "s3://" + target_bucket + "/datasets/phi-3/"


## Pipeline Parameters

In [4]:
# 定義參數
training_datasets_s3_uri = ParameterString(name="TrainingDatasetesS3Uri", default_value=datasets_s3_uri)
inference_instance_type = ParameterString(name="InferenceInstanceType", default_value="ml.g5.xlarge")


# 其他配置
model_package_group_name = "Demo-SageMaker-Pipeline-Group"
cache_config = CacheConfig(enable_caching=True, expire_after="30d")

## TraningStep

In [5]:
from sagemaker.huggingface import HuggingFace
from transformers import AutoTokenizer

model_id = "microsoft/Phi-3.5-mini-instruct"



# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'num_train_epochs': 3,                            # number of training epochs
  'per_device_train_batch_size': 1,                 # batch size for training
  'gradient_accumulation_steps': 2,                 # Number of updates steps to accumulate 
  'gradient_checkpointing': True,                   # save memory but slower backward pass
  'fp16': True ,
  'learning_rate': 2e-4,                            # learning rate
  'max_grad_norm': 0.3,                             # Maximum norm (for gradient clipping)
  'warmup_ratio': 0.03,                             # warmup ratio
  "lr_scheduler_type":"constant",                   # learning rate scheduler
  'save_strategy': "epoch",                         # save strategy for checkpoints
  "logging_steps": 10,                              # log every x steps
  'merge_adapters': True,                           # wether to merge LoRA into the model (needs more memory)
  'use_flash_attn': True,                           # Whether to use Flash Attention
  'output_dir': '/tmp/run',                         # output directory, where to save assets during training
}

# define Training Job Name 
job_name = f'huggingface-qlora-{hyperparameters["model_id"].replace("/","-").replace(".","-")}'


huggingface_estimator = HuggingFace(
    entry_point          = 'run_qlora.py',    # train script
    source_dir           = '../scripts',      # directory which includes all the files needed for training
    instance_type        = 'ml.p3.2xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 2*24*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 300,               # the size of the EBS volume in GB
    transformers_version = '4.36',            # the transformers version used in the training job
    pytorch_version      = '2.1',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
)


# 定義訓練步驟
train_step = TrainingStep(
    name="TrainModel",
    estimator=huggingface_estimator,
    inputs={
        'training': TrainingInput(training_datasets_s3_uri, content_type="application/json")
    },
    cache_config=cache_config
)


## Register Model Step
Documentation: https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-register-model

In [6]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
)

config = {
    'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
    'SM_NUM_GPUS': json.dumps(1), # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

model = HuggingFaceModel( # https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model
    model_data=train_step.properties.ModelArtifacts.S3ModelArtifacts, # https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html
    role=role,
    image_uri=llm_image,
    sagemaker_session=sess,
    env=config
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=[inference_instance_type],
    model_package_group_name=model_package_group_name,
    approval_status="Approved",
)

## Lambda Step for Deploying model endpoint
documents: 
- https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda
- https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker/client/create_model.html

In [7]:
%%writefile lambda_deployer.py

import json

import boto3

import time

# Use the current time to define unique names for the resources created
current_time = time.strftime("%H-%M", time.localtime())


def lambda_handler(event, context):
    """Lambda function to deploy a model to an Endpoint using boto3 and create an API Gateway for SageMaker"""

    # 使用 event 中的參數
    model_package_arn = event["model_package_arn"]
    model_name = event["model_name"] + "-run-at-" + current_time
    endpoint_config_name = event["endpoint_config_name"]
    endpoint_name = event["endpoint_name"]
    role = event["role"]
    apigateway_role = event["apigateway_role"]
    inference_instance_type = event["inference_instance_type"]

    sm_client = boto3.client("sagemaker")
    apigateway_client = boto3.client("apigateway")

    # 創建 SageMaker 模型
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        PrimaryContainer={"ModelPackageName": model_package_arn},
        ExecutionRoleArn=role,
    )

    # 創建端點配置
    create_endpoint_config_response = sm_client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "InstanceType": inference_instance_type,
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "ModelName": model_name,
                "VariantName": "AllTraffic",
            }
        ],
    )

    # 創建端點
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
    )

    # 創建 API Gateway
    api_name = f"{model_name}-api"
    create_api_response = apigateway_client.create_rest_api(
        name=api_name,
        description=f"API Gateway for SageMaker endpoint {endpoint_name}",
        endpointConfiguration={"types": ["REGIONAL"]},
    )

    # 取得 API Gateway 的根資源 ID
    api_id = create_api_response["id"]
    root_id = apigateway_client.get_resources(restApiId=api_id)["items"][0]["id"]

    # 創建 POST 方法並設定
    apigateway_client.put_method(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        authorizationType="NONE",
    )

    # 設置 SageMaker Runtime 與 API Gateway 的整合
    apigateway_client.put_integration(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        type="AWS",  # 使用 AWS Service Integration
        integrationHttpMethod="POST",
        uri=f"arn:aws:apigateway:{boto3.Session().region_name}:runtime.sagemaker:path/endpoints/{endpoint_name}/invocations",
        credentials=apigateway_role,  # 指定具有 SageMaker InvokeEndpoint 權限的角色
    )

    # 設置 Integration Response，以便 API Gateway 正確處理 SageMaker 回應
    apigateway_client.put_integration_response(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        statusCode="200",
        responseTemplates={"application/json": "$input.body"},
    )

    # 創建方法回應，確保有適當的狀態碼
    apigateway_client.put_method_response(
        restApiId=api_id,
        resourceId=root_id,
        httpMethod="POST",
        statusCode="200",
        responseModels={"application/json": "Empty"},
    )

    # 部署 API Gateway
    apigateway_client.create_deployment(
        restApiId=api_id,
        stageName="dev",
    )

    # 返回 API Gateway 的 URL
    api_url = (
        f"https://{api_id}.execute-api.{boto3.Session().region_name}.amazonaws.com/dev"
    )

    return {
        "statusCode": 200,
        "body": json.dumps(
            {
                "message": f"Created Endpoint {endpoint_name} and API Gateway {api_name}!",
                "endpoint_name": endpoint_name,
                "api_url": api_url,
            }
        ),
    }


In [11]:
import boto3
import json

iam = boto3.client('iam')

def attach_policy_if_missing(role_name, policy_arn):
    """檢查並附加策略到角色，若策略已存在則跳過"""
    try:
        # 獲取附加到角色的現有策略
        attached_policies = iam.list_attached_role_policies(RoleName=role_name)['AttachedPolicies']
        attached_policy_arns = [policy['PolicyArn'] for policy in attached_policies]

        # 如果策略不在已附加列表中，則附加
        if policy_arn not in attached_policy_arns:
            iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
            print(f"Attached policy {policy_arn} to role {role_name}")
        else:
            print(f"Policy {policy_arn} already attached to role {role_name}")

    except Exception as e:
        print(f"Error attaching policy {policy_arn} to role {role_name}: {e}")

def create_lambda_role(role_name, apigateway_role_arn):
    try:
        # 創建 IAM 角色
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for Lambda to interact with SageMaker, API Gateway, and Bedrock'
        )

        role_arn = response['Role']['Arn']

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']

    # 無論角色是否已存在，都會檢查並附加所需的策略
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator')

    # 附加 Lambda Function URL 創建的策略
    policy_document_url_config = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "lambda:CreateFunctionUrlConfig",
                "Resource": f"arn:aws:lambda:*:{role_arn.split(':')[4]}:function:*"
            }
        ]
    }
    
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="LambdaURLConfigPolicy",
        PolicyDocument=json.dumps(policy_document_url_config)
    )

    # 在 Lambda 角色中附加 iam:PassRole 權限，允許傳遞 API Gateway 的角色
    policy_document_pass_role = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "iam:PassRole",
                "Resource": apigateway_role_arn
            }
        ]
    }

    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="PassRolePolicy",
        PolicyDocument=json.dumps(policy_document_pass_role)
    )

    # 附加 Bedrock 權限
    bedrock_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:ListModels",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": "*"
            }
        ]
    }

    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="BedrockInvokePolicy",
        PolicyDocument=json.dumps(bedrock_policy_document)
    )

    print(f"Attached iam:PassRole, lambda:CreateFunctionUrlConfig, and Bedrock policies to {role_name}")

    return role_arn


def create_apigateway_role(role_name):
    try:
        # 創建 API Gateway 用的 IAM 角色
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "apigateway.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description='Role for API Gateway to invoke SageMaker Endpoints'
        )

        role_arn = response['Role']['Arn']

    except iam.exceptions.EntityAlreadyExistsException:
        print(f'Using existing role: {role_name}')
        response = iam.get_role(RoleName=role_name)
        role_arn = response['Role']['Arn']

    # 附加允許 API Gateway 調用 SageMaker 的策略
    attach_policy_if_missing(role_name, 'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess')

    return role_arn

In [12]:
import time
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
apigateway_role = create_apigateway_role("apigateway-role")
lambda_role = create_lambda_role("lambda-deployment-role", apigateway_role_arn=apigateway_role)


# Use the current time to define unique names for the resources created
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

model_name = "demo-sagemaker-pipeline-model" + current_time
endpoint_name = "demo-sagemaker-pipeline-endpoint-" + current_time
endpoint_config_name = "demo-sagemaker-pipeline-endpoint-config" + current_time
function_name = "demo-sagemaker-pipeline-lambda-step" + current_time

# Lambda helper class can be used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="lambda_deployer.py",
    handler="lambda_deployer.lambda_handler",
    timeout=600,
    memory_size=10240,
)

# The dictionary retured by the Lambda function is captured by LambdaOutput, each key in the dictionary corresponds to a
# LambdaOutput

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)

# The inputs provided to the Lambda function can be retrieved via the `event` object within the `lambda_handler` function
# in the Lambda
lambda_deploy_step = LambdaStep(
    name="LambdaStepDeployModelEndpoint",
    lambda_func=func,
    inputs={
        "model_package_arn": register_step.properties.ModelPackageArn,
        "model_name": model_name,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
        "role": role,
        "apigateway_role": apigateway_role,
        "inference_instance_type": inference_instance_type
    },
    outputs=[output_param_1, output_param_2])

## CreateStreamingResponseLambdaFunction

In [13]:
%%writefile main.py

import json
import os
from typing import List, Optional

import boto3
import uvicorn
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import RedirectResponse, StreamingResponse
from pydantic import BaseModel


app = FastAPI()

bedrock = boto3.client("bedrock-runtime")
sagemaker_runtime = boto3.client("sagemaker-runtime")

SAGEMAKER_ENDPOINT_NAME = os.getenv("SAGEMAKER_ENDPOINT_NAME")

# CORS 設定
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.mount("/demo", StaticFiles(directory="static", html=True))

@app.get("/")
async def root():
    return RedirectResponse(url="/demo/")

class Message(BaseModel):
    role: str  # Role can be 'user' or 'assistant'
    content: str  # The content of the message

class ChatRequest(BaseModel):
    model: str  # Model name provided by the client
    system: Optional[str] = None  # Optional system prompt
    messages: List[Message]  # List of messages with roles and content
    temperature: Optional[float] = 0.5  # Optional, default temperature is 0.5
    max_tokens: Optional[int] = 1024  # Optional, default max_tokens is 1024
    stream: Optional[bool] = True  # Enable streaming by default


@app.post("/v1/chat/completions")
def api_chat_completion(chat_request: ChatRequest):
    if not chat_request.messages:
        return {"error": "Messages are required"}

    # 如果 model 是 'psy-1'，調用 SageMaker 端點
    if chat_request.model == "psy-1":
        body = {
            "inputs": f"<|system|>{chat_request.system or ''}<|user|>{chat_request.messages[-1].content}<|assistant|>",
            "parameters": {
                "do_sample": True,
                "top_p": 0.9,
                "temperature": chat_request.temperature,
                "max_new_tokens": chat_request.max_tokens,
                "repetition_penalty": 1.03,
                "stop": ["\nUser:", "<|endoftext|>", "###"],
            },
            "stream": chat_request.stream,
        }
        return StreamingResponse(
            sagemaker_stream(
                SAGEMAKER_ENDPOINT_NAME, body
            ),
            media_type="text/html",
        )

    # Default to Bedrock model
    body = {
        "max_tokens": chat_request.max_tokens,  # Accept max_tokens from the front-end
        "anthropic_version": "bedrock-2023-05-31",  # Required by Bedrock API
        "messages": [
            {"role": msg.role, "content": msg.content} for msg in chat_request.messages
        ],
        "temperature": chat_request.temperature,  # Accept temperature from the front-end
    }

    # Include the system prompt if provided
    if chat_request.system:
        body["system"] = chat_request.system

    return StreamingResponse(
        bedrock_stream(chat_request.model, body), media_type="text/html"
    )

async def bedrock_stream(model_id: str, body: dict):
    # Convert the dictionary into a JSON string
    body_str = json.dumps(body)

    # Send the model ID from the request and the body to Bedrock
    response = bedrock.invoke_model_with_response_stream(
        modelId=model_id,  # Model name provided in the request body
        body=body_str,
    )

    stream = response.get("body")
    if stream:
        for event in stream:
            chunk = event.get("chunk")
            if chunk:
                message = json.loads(chunk.get("bytes").decode())
                if message["type"] == "content_block_delta":
                    # Stream the content back to the client
                    yield message["delta"]["text"] or ""
                elif message["type"] == "message_stop":
                    # Indicate the end of the message
                    yield "\n"

async def sagemaker_stream(endpoint_name: str, body: dict):
    body_str = json.dumps(body)

    response = sagemaker_runtime.invoke_endpoint_with_response_stream(
        EndpointName=endpoint_name,
        Body=body_str,
        ContentType='application/json'
    )

    stream = response['Body']
    complete_text = ""  # 用來存儲最終的合併文本
    incomplete_message = ""  # 用來存儲不完整的消息

    if stream:
        for event in stream:
            if 'PayloadPart' in event:
                try:
                    raw_message = event['PayloadPart']['Bytes']
                    # 設定解碼錯誤策略，允許跳過無效字元
                    decoded_message = raw_message.decode('utf-8', errors='ignore')
                    print(f"Raw Message after stripping 'data:': {decoded_message}")

                    if decoded_message.startswith("data:"):
                        decoded_message = decoded_message[5:].strip()

                    # 將不完整的訊息拼接起來
                    if incomplete_message:
                        decoded_message = incomplete_message + decoded_message
                        incomplete_message = ""

                    # 嘗試解析為 JSON
                    try:
                        json_message = json.loads(decoded_message)
                    except json.JSONDecodeError:
                        # 如果 JSON 解析失敗，將訊息暫存並等待下一個 event
                        incomplete_message = decoded_message
                        continue

                    # 確認 token 存在且非特殊字符
                    token = json_message.get('token', {}).get('text')
                    special = json_message.get('token', {}).get('special', False)

                    # 檢查 token 是否是 "<|end|>" 或 special 為 true，直接終止流
                    if token == "<|end|>" or special:
                        print("End token detected, stopping stream.")
                        break  # 結束迴圈並停止拼接

                    if token and token.strip():
                        complete_text += token  # 拼接完整的 text
                        yield token  # 實時回傳 token

                except json.JSONDecodeError:
                    print("JSON Decode Error: Skipping invalid JSON data.")
                    continue

            elif 'ModelStreamError' in event:
                error_message = event['ModelStreamError']['Message']
                print(f"Model stream error: {error_message}")
                # 過濾錯誤訊息，不傳回給前端
                continue

            elif 'InternalStreamFailure' in event:
                print(f"Internal stream failure: {event['InternalStreamFailure']['Message']}")
                # 過濾錯誤訊息，不傳回給前端
                continue



if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=int(os.environ.get("PORT", "8080")))



In [14]:
%%writefile run.sh

#!/bin/bash

PATH=$PATH:$LAMBDA_TASK_ROOT/bin \
    PYTHONPATH=$PYTHONPATH:/opt/python:$LAMBDA_RUNTIME_DIR \
    exec python -m uvicorn --port=$PORT main:app


In [15]:
!mkdir -p static

In [19]:
%%writefile static/index.html

<!DOCTYPE html>
<html>
  <head>
    <title>Psy test demo</title>
    <link rel="stylesheet" href="style.css" />
    <link
      rel="stylesheet"
      href="https://fonts.googleapis.com/css?family=Roboto:300,300italic,700,700italic"
    />
    <link
      rel="stylesheet"
      href="https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.css"
    />
    <link
      rel="stylesheet"
      href="https://cdnjs.cloudflare.com/ajax/libs/milligram/1.4.1/milligram.css"
    />
  </head>

  <body>
    <div id="container" class="row">
      <div class="column column-67">
        <h1>Psy test demo</h1>
        <h4>
          Enter your request details below and let the AI generate a response.
        </h4>

        <!-- Input fields for model, system prompt, and user message -->
        <label for="model">Model:</label>
        <small
          ><a
            href="https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html"
            target="_blank"
            >Refer to the AWS Bedrock model IDs documentation</a
          ></small
        >
        <input
          type="text"
          id="model"
          placeholder="Enter model ID (e.g., 'anthropic.claude-3-sonnet-20240229-v1:0')"
          value="anthropic.claude-3-sonnet-20240229-v1:0"
        />

        <label for="system">System Prompt:</label>
        <textarea id="system" placeholder="Enter system prompt"></textarea>

        <label for="user-message">User Message:</label>
        <textarea id="user-message" placeholder="Enter user message"></textarea>

        <!-- Input for max_tokens and temperature -->
        <label for="max-tokens">Max Tokens:</label>
        <input
          type="number"
          id="max-tokens"
          value="1024"
          min="1"
          placeholder="Enter max tokens"
        />

        <label for="temperature">Temperature:</label>
        <input
          type="number"
          step="0.1"
          id="temperature"
          value="0.5"
          min="0"
          max="1"
          placeholder="Enter temperature"
        />

        <!-- Button to trigger the API call -->
        <button id="generate-response">Generate</button>

        <!-- Output area for the story -->
        <div id="story-output"></div>
      </div>
    </div>

    <script src="script.js"></script>
  </body>
</html>


In [20]:
%%writefile static/script.js

async function generateAIResponse() {
  // Get input values from the form
  const model = document.getElementById("model").value;
  const system = document.getElementById("system").value;
  const userMessage = document.getElementById("user-message").value;
  const maxTokens = document.getElementById("max-tokens").value;
  const temperature = document.getElementById("temperature").value;

  if (userMessage.trim().length === 0) {
    return;
  }

  const storyOutput = document.getElementById("story-output");
  storyOutput.innerText = "Thinking...";

  try {
    // Create request payload
    const requestBody = {
      model: model,
      system: system,
      messages: [{
        role: 'user',
        content: userMessage
      }],
      max_tokens: parseInt(maxTokens),
      temperature: parseFloat(temperature),
      stream: true
    };

    // Use Fetch API to send a POST request for response streaming
    const response = await fetch("/v1/chat/completions", {
      method: "POST",
      headers: {
        "Content-Type": "application/json"
      },
      body: JSON.stringify(requestBody)
    });

    storyOutput.innerText = "";

    // Response Body is a ReadableStream
    const reader = response.body.getReader();
    const decoder = new TextDecoder();

    // Process the chunks from the stream
    while (true) {
      const {
        done,
        value
      } = await reader.read();
      if (done) {
        break;
      }
      const text = decoder.decode(value);
      storyOutput.innerText += text;
    }

  } catch (error) {
    storyOutput.innerText = `Sorry, an error happened. Please try again later. \n\n ${error}`;
  }
}

document.getElementById("generate-response").addEventListener("click", generateAIResponse);
document.getElementById('user-message').addEventListener('keydown', function (e) {
  if (e.code === 'Enter') {
    generateAIResponse();
  }
});

In [21]:
%%writefile static/style.css

body {
    font-family: sans-serif;
    margin: 0;
    padding: 0;
  }
  
  #container {
    justify-content: center
  }
  
  h1 {
    text-align: center;
  }
  
  p {
    margin-bottom: 10px;
  }
  
  input {
    width: 100%;
    height: 20px;
    border: 1px solid black;
    margin-bottom: 10px;
  }
  
  button {
    height: 20px;
    background-color: #000;
    color: #fff;
    border: none;
    cursor: pointer;
  }
  
  #story-output {
    width: 100%;
    overflow: auto;
  }
  

In [22]:
import os
import zipfile
import shutil

# Define the names of your files and directories
main_script = 'main.py'
run_script = 'run.sh'
static_dir = 'static'
zip_filename = 'lambda_package.zip'

# Step 1: Create a temporary directory to store the files
if not os.path.exists('lambda_temp'):
    os.mkdir('lambda_temp')

# Step 2: Copy your scripts (main.py and run.sh) into the temp directory
shutil.copy(main_script, './lambda_temp/')
shutil.copy(run_script, './lambda_temp/')

# Step 3: Copy the 'static' directory into the temp directory
if os.path.exists(static_dir):
    shutil.copytree(static_dir, './lambda_temp/static')

# Step 4: Create the .zip package
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk('lambda_temp'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, 'lambda_temp'))

# Step 5: Clean up the temp directory
shutil.rmtree('lambda_temp')

print(f"{zip_filename} created successfully.")


In [26]:
import os
import boto3
import subprocess
import zipfile
import shutil

# 定義變數
layer_dir = "./layer/python"
zip_filename = "layer.zip"
region = sess.boto_region_name
layer_name = "fast_api_related_lambda_layer"
layer_description = "Lambda layer for FastAPI with Lambda Web Adapter"

# Step 1: 建立 layer/python 資料夾
if not os.path.exists(layer_dir):
    os.makedirs(layer_dir)

# Step 2: 安裝指定的套件到 layer/python 資料夾
subprocess.run([
    "pip3", "install", "--target", layer_dir, 
    "annotated-types==0.6.0", 
    "anyio==4.2.0", 
    "click==8.1.7", 
    "exceptiongroup==1.2.0", 
    "fastapi==0.109.2", 
    "h11==0.14.0", 
    "idna==3.7", 
    "pydantic==2.6.1", 
    "pydantic_core==2.16.2", 
    "sniffio==1.3.0", 
    "starlette==0.36.3", 
    "typing_extensions==4.9.0", 
    "uvicorn==0.27.0.post1"
], check=True)

# Step 3: 將 layer 目錄壓縮成 .zip 檔案
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk('./layer'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=os.path.relpath(file_path, './layer'))



print(f"{zip_filename} created successfully.")


# 上傳成 Lambda Layer
lambda_client = boto3.client('lambda', region_name=region)

with open(zip_filename, 'rb') as f:
    response = lambda_client.publish_layer_version(
        LayerName=layer_name,
        Description=layer_description,
        Content={'ZipFile': f.read()},
        CompatibleRuntimes=['python3.11'],  # 根據你的 Lambda 版本設定
        LicenseInfo='MIT'  # 可選擇性設定 license
    )

fastapi_related_layer_arn = response["LayerVersionArn"]

# 顯示 Layer ARN
print("Layer uploaded successfully:")
print(fastapi_related_layer_arn)

# 清理 layer 資料夾
shutil.rmtree('./layer')


In [27]:
import time
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.workflow.functions import JsonGet # https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.functions.JsonGet

# Use the current time to define unique names for the resources created
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

streaming_response_function_name="lambda_streaming_response_function-" + current_time

lambda_web_adpter_layer_arn = f"arn:aws:lambda:{sess.boto_region_name}:753240598075:layer:LambdaAdapterLayerX86:23"
pydantic_layer_arn = "arn:aws:lambda:us-west-2:770693421928:layer:Klayers-p311-pydantic:10"

# 創建 Lambda 函數物件
lambda_function = Lambda(
    function_name=streaming_response_function_name,
    execution_role_arn=lambda_role,
    zipped_code_dir="lambda_package.zip",
    handler="run.sh",
    timeout=600,
    memory_size=10240,
    runtime="python3.11",
    environment={
        "Variables": {  
            "SAGEMAKER_ENDPOINT_NAME": endpoint_name,
            "AWS_LAMBDA_EXEC_WRAPPER": "/opt/bootstrap",
            "AWS_LWA_INVOKE_MODE": "response_stream",
            "PORT": "8000"
        }
    },
    layers=[lambda_web_adpter_layer_arn, pydantic_layer_arn, fastapi_related_layer_arn]
)

# 創建 Lambda Step
lambda_create_streaming_response_step = LambdaStep(
    name="CreateStreamingResponseLambdaFunction",
    lambda_func=lambda_function,
    depends_on=[lambda_deploy_step]
)


## CreateLambdaFunctionURL Step

In [28]:
%%writefile lambda_create_function_url.py

import boto3

def lambda_handler(event, context):
    lambda_client = boto3.client('lambda')
    function_name = event['function_name']
    
    # 創建 Lambda Function URL
    response = lambda_client.create_function_url_config(
        FunctionName=function_name,
        AuthType='NONE',
        InvokeMode='RESPONSE_STREAM'
    )
    
    # 返回 Function URL
    return {
        'statusCode': 200,
        'body': {
            'function_url': response['FunctionUrl']
        }
    }

In [29]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum


# 創建 Lambda 函數，用於創建 Function URL
lambda_create_function_url = Lambda(
    function_name="create_lambda_function_url",
    execution_role_arn=lambda_role,
    script="lambda_create_function_url.py",
    handler="lambda_create_function_url.lambda_handler",
    timeout=300,
    memory_size=128
)

# 創建 LambdaStep
create_lambda_function_url_step = LambdaStep(
    name="CreateLambdaFunctionURL",
    lambda_func=lambda_create_function_url,
    inputs={
        "function_name": streaming_response_function_name
    },
    outputs=[
        LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String),
        LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
    ],
    depends_on=[lambda_create_streaming_response_step]
)


## Pipeline Definition

In [31]:
# 定義 Pipeline
pipeline = Pipeline(
    name="Training-Register-Deploy-ExposeEndpoint-Pipeline",
    parameters=[training_datasets_s3_uri, inference_instance_type],
    steps=[train_step, register_step, lambda_deploy_step, lambda_create_streaming_response_step, create_lambda_function_url_step]
)

# 更新 SageMaker Pipeline (不存在則創建)
pipeline.upsert(role_arn=role)


In [ ]:
# # 取得執行狀態
# execution.describe()

# # 等待 Pipeline 執行完成
# execution.wait()


## Manually Deploy a Model from the Registry
Domumentation: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html#model-registry-deploy-smsdk

In [ ]:
from sagemaker import ModelPackage # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage
from time import gmtime, strftime

model_package_arn = "arn:aws:sagemaker:us-west-2:097724924093:model-package/Demo-SageMaker-Pipeline-Group/3"
model = ModelPackage(role=role, # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage
                     model_package_arn=model_package_arn,
                     sagemaker_session=sess)
model.deploy(initial_instance_count=1, instance_type='ml.g5.xlarge', container_startup_health_check_timeout=1000) # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

## Manually Deploy a Model from S3 (uncompressed)

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

# Model s3 prefix(folder) uri  
model_s3_prefix_uri = "s3://sagemaker-us-west-2-097724924093/huggingface-qlora-microsoft-Phi-3-5-min-2024-09-11-04-03-17-319/output/model/"

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
)

config = {
    'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
    'SM_NUM_GPUS': json.dumps(1), # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

model = HuggingFaceModel( # https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model
    model_data={'S3DataSource':{'S3Uri': model_s3_prefix_uri,'S3DataType': 'S3Prefix','CompressionType': 'None'}}, # We use a dict, for more details, please refer to these two documents: https://sagemaker.readthedocs.io/en/stable/api/inference/model.html, https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3ModelDataSource.html
    role=role,
    image_uri=llm_image,
    sagemaker_session=sess,
    env=config
)


model.deploy(initial_instance_count=1, instance_type='ml.g5.2xlarge', container_startup_health_check_timeout=1000) # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy